In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Dense
from keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks

In [5]:
train_images_array = np.load("train_data/train_images_full.npy")
train_labels_array = np.load("train_data/train_labels_full.npy")

In [6]:
train_images_array.shape

(15677, 224, 224, 3)

In [7]:
train_labels_array = to_categorical(train_labels_array, dtype="int")

In [8]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

2022-06-12 13:49:20.599781: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
for layers in (vggmodel.layers)[:15]:
    layers.trainable = False

In [10]:
dense_layer2 = vggmodel.layers[-2].output
prediction_layer = Dense(units=2, activation="softmax")(dense_layer2)

vgecocup = Model(inputs=vggmodel.input, outputs=prediction_layer)
optimizer = Adam(learning_rate=0.0001)
vgecocup.compile(optimizer=optimizer,
                 loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
vgecocup.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(
    train_images_array, train_labels_array, test_size=0.10)

In [12]:
X_train_model.shape

(14109, 224, 224, 3)

In [13]:
X_test_model.shape

(1568, 224, 224, 3)

In [15]:
hist = vgecocup.fit(
    x=X_train_model,
    y=y_train_model,
    verbose=1,
    batch_size=10,
    steps_per_epoch=10,
    epochs=150,
    validation_data=(X_test_model, y_test_model),
)

Epoch 1/150
10/10 [==============================] - 442s 48s/step - loss: 0.3939 - accuracy: 0.8800 - val_loss: 0.4023 - val_accuracy: 0.9037
Epoch 2/150
10/10 [==============================] - 410s 45s/step - loss: 0.5022 - accuracy: 0.8800 - val_loss: 0.2268 - val_accuracy: 0.9209
Epoch 3/150
10/10 [==============================] - 423s 47s/step - loss: 0.2878 - accuracy: 0.9000 - val_loss: 0.2432 - val_accuracy: 0.9075
Epoch 4/150
10/10 [==============================] - 440s 48s/step - loss: 0.3903 - accuracy: 0.9100 - val_loss: 0.6928 - val_accuracy: 0.8080
Epoch 5/150
10/10 [==============================] - 450s 50s/step - loss: 0.3512 - accuracy: 0.8600 - val_loss: 0.1522 - val_accuracy: 0.9394
Epoch 6/150
10/10 [==============================] - 490s 54s/step - loss: 0.1891 - accuracy: 0.9400 - val_loss: 0.1562 - val_accuracy: 0.9432
Epoch 7/150
10/10 [==============================] - 523s 58s/step - loss: 0.0551 - accuracy: 0.9800 - val_loss: 0.3781 - val_accuracy: 0.9222